In [ ]:
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/Noha-Magdy/segment-anything.git'

! wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
!pip install monai

  Cloning https://github.com/Noha-Magdy/segment-anything.git to /tmp/pip-req-build-00cm2msj
  Running command git clone --filter=blob:none --quiet https://github.com/Noha-Magdy/segment-anything.git /tmp/pip-req-build-00cm2msj
  Resolved https://github.com/Noha-Magdy/segment-anything.git to commit 98439a88601f4290b63152ab48b1efc88e4c6cd9
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36606 sha256=8a7e64218d8d178f65cb799d19a097841d2b227e2ef09d1e56939ca312b53174
  Stored in directory: /tmp/pip-ephem-wheel-cache-4mftp2nl/wheels/a5/4a/ea/b9ae92897766fc687a9f74ac9c0351adfd61e2df1e37cb2f16
Successfully built segment-anything
--2023-06-28 06:23:07--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.8.51, 13.35.8.29, 13.35.8.19, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.8.51|:443... connected.
HTT

## Pipline for saved 2d data

In [ ]:
# saved 2d data

print("No bounding box, flair")
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import Subset

from tqdm import tqdm
from torchsummary import summary
import monai
from segment_anything import SamPredictor, sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide
# import wandb
import nibabel as nib
from segment_anything.utils.transforms import ResizeLongestSide
import torch.nn.functional as F
from skimage import transform, io, segmentation


torch.manual_seed(2023)
np.random.seed(2023)

# change here
data_path = "/content/out"


def squarify(M,val = 0):
    (a,b)=M.shape
    if a>b:
        padding=((0,0),(0,a-b))
    else:
        padding=((0,b-a),(0,0))
    return np.pad(M,padding,mode='constant',constant_values=val)



def get_bounding_box(mask):
  x2 = mask.shape[0]
  y2 = mask.shape[1]

  return [0, 0, x2, y2]


def dice_loss(y_true, y_pred):
    """
    implementation for generalized dice score loss
    GDL = 1 - (2 * sum(intersection)/(sum(pred)+sum(true)))
    """

    # y_pred = tf.round(y_pred)
    intersection = torch.sum(y_true * y_pred)
    pred_sum = torch.sum( y_pred)
    true_sum = torch.sum( y_true)
    smooth = torch.ones_like(intersection)*1e-5
    return 1-((2*intersection+smooth)/(pred_sum+true_sum+smooth))


model_type = 'vit_h'
checkpoint = 'sam_vit_h_4b8939.pth'
sam_model = sam_model_registry[model_type](checkpoint=checkpoint).to('cuda')



embedding_path = "/content/BraTS_data/flair/z/embeddings"
ground_truth_path = "/content/BraTS_data/mask/z"

train_list = os.listdir(embedding_path)
train_list = train_list[:int(len(train_list)*0.7)]
val_list = train_list[int(len(train_list)*0.7):]

class BraTSDataset(Dataset):

    def __init__(self, ground_truth_path, embedding_path, list_of_data, device='cuda'):
        """
        Args:
            embedding_path (string): Path to  images.
            masks_path (string): path to masks.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.ground_truth_path = ground_truth_path
        self.device = device
        self.list_of_data = list_of_data
        self.embedding_path = embedding_path

    def __len__(self):
        return len(self.list_of_data)

    def __getitem__(self, idx):


      embedding = np.load(self.embedding_path + "/" + self.list_of_data[idx])
      embedding = torch.as_tensor(embedding, dtype=torch.float, device=self.device)


      #--------------------------------------------------

      mask = np.load(self.ground_truth_path + "/" + self.list_of_data[idx])
      mask[mask>0] = 1
      box = get_bounding_box(mask)
      # mask = squarify(mask)



      mask = transform.resize(
                      mask,
                      (256, 256),
                      order=0,
                      preserve_range=True,
                      mode="constant",
                  )


        # class_1 = np.zeros_like(mask)
        # class_2 = np.zeros_like(mask)
        # class_3 = np.zeros_like(mask)

        # class_1[np.where(mask == 1)] = 1
        # class_2[np.where(mask == 2)] = 1
        # class_3[np.where(mask == 4)] = 1

        # mask = mask = np.stack((class_1, class_2, class_3), axis = -1 )
      mask = torch.as_tensor(mask).to(self.device)
      mask = mask[None,...]


      box = get_bounding_box(mask)
      # box = [0,0,mask.shape[-2],mask.shape[-1]]
      box_np = np.array(box)
      sam_trans = ResizeLongestSide(sam_model.image_encoder.img_size)
      box = sam_trans.apply_boxes(box_np, (mask.shape[-2], mask.shape[-1]))
      box_torch = torch.as_tensor(box, dtype=torch.float, device=self.device)


      sample = {"embedding": embedding,
                  'mask': mask,
                  'box' : box_torch
                  }

      return sample


train = BraTSDataset(ground_truth_path, embedding_path, train_list)
val = BraTSDataset(ground_truth_path, embedding_path, val_list)



train_loader = DataLoader(train, batch_size= 8, shuffle=True, num_workers=0)
val_loader = DataLoader(val, batch_size= 4, shuffle=True, num_workers=0)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



hprams = {
    "num_epochs": 15,
    "best_loss": 1e10,
    "model_save_path": "/content",
    "lr":1e-5,
    "weight_decay":0,
    "device": device,
    "train_dataloader": train_loader,
    "val_dataloader": val_loader
    }

def eval(val_dataloader, sam_model, seg_loss):
  loss = 0
  for step_val, data in enumerate(val_dataloader):
    embedding, mask, box = data["embedding"], data["mask"], data["box"]

    with torch.no_grad():
      # embedding = sam_model.image_encoder(image)
      sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
          points=None,
          boxes=box,
          masks=None,
      )

      # predicted masks
      mask_predictions, _ = sam_model.mask_decoder(
      image_embeddings= embedding.to(device), # (B, 256, 64, 64)
      image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
      sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
      dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
      multimask_output=False,
      )

      loss += seg_loss(mask_predictions, mask.to(device))
  loss /= step_val+1
  return loss



def train(**kwargs):
    model_save_path = kwargs['model_save_path']
    num_epochs = kwargs['num_epochs']
    best_loss = kwargs['best_loss']
    lr = kwargs['lr']
    weight_decay = kwargs['weight_decay']
    device = kwargs['device']
    train_dataloader = kwargs['train_dataloader']
    # embedding_path = kwargs['embedding_path']
    val_dataloader = kwargs['val_dataloader']

    os.makedirs(model_save_path, exist_ok=True)

    sam_model.train()
    optimizer = torch.optim.Adam(sam_model.mask_decoder.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, factor=0.25,  patience=5)
    seg_loss = monai.losses.DiceLoss(sigmoid=True, squared_pred=True, reduction='mean')


    for epoch in range(num_epochs):
        epoch_loss = 0
        # train
        for step, data in enumerate(tqdm(train_dataloader)):
            mask, box, embedding =  data["mask"], data["box"], data["embedding"]
            embedding.to(device)

            with torch.no_grad():

              sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
                  points=None,
                  boxes=box,
                  masks=None,
              )

            # predicted masks
            mask_predictions, _ = sam_model.mask_decoder(
              image_embeddings= embedding.to(device), # (B, 256, 64, 64)
              image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
              sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
              dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
              multimask_output=False,
            )

            loss = seg_loss(mask_predictions, mask.to(device))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            # break
        epoch_loss /= step+1
        scheduler.step(epoch_loss)
        val_loss = None
        if val_dataloader is not None: val_loss = eval(val_dataloader, sam_model, seg_loss)
        print(f'EPOCH: {epoch}, Loss: {epoch_loss}, Val loss {val_loss}')
        # wandb.log({"epoch": epoch, "loss": loss})

        # save the latest model checkpoint
        # torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_'+str(epoch)+'.pth'))

        # save the best model
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_best.pth'))




train(**hprams)

No bounding box, flair


FileNotFoundError: ignored

## pipeline for saved 2d embeddings and 3d masks (no bounding box)

In [ ]:
# 2d input 3d mask

print("No bounding box, flair_z")
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import Subset

from tqdm import tqdm
from torchsummary import summary
import monai
from segment_anything import SamPredictor, sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide
# import wandb
import nibabel as nib
from segment_anything.utils.transforms import ResizeLongestSide
import torch.nn.functional as F
from skimage import transform, io, segmentation


torch.manual_seed(2023)
np.random.seed(2023)

# change here
data_path = "/content/out"


def squarify(M,val = 0):
    (a,b)=M.shape
    if a>b:
        padding=((0,0),(0,a-b))
    else:
        padding=((0,b-a),(0,0))
    return np.pad(M,padding,mode='constant',constant_values=val)



def get_bounding_box(mask):
  x2 = mask.shape[0]
  y2 = mask.shape[1]

  return [0, 0, x2, y2]


def dice_loss(y_true, y_pred):
    """
    implementation for generalized dice score loss
    GDL = 1 - (2 * sum(intersection)/(sum(pred)+sum(true)))
    """

    # y_pred = tf.round(y_pred)
    intersection = torch.sum(y_true * y_pred)
    pred_sum = torch.sum( y_pred)
    true_sum = torch.sum( y_true)
    smooth = torch.ones_like(intersection)*1e-5
    return 1-((2*intersection+smooth)/(pred_sum+true_sum+smooth))


model_type = 'vit_h'
checkpoint = 'sam_vit_h_4b8939.pth'
sam_model = sam_model_registry[model_type](checkpoint=checkpoint).to('cuda')



embedding_path = "/content/BraTS_data/flair/z/embeddings"
ground_truth_path = "/content/out"

train_list = os.listdir(embedding_path)
train_list = train_list[:int(len(train_list)*0.7)]
val_list = train_list[int(len(train_list)*0.7):]

class BraTSDataset(Dataset):

    def __init__(self, ground_truth_path, embedding_path, list_of_data, device='cuda'):
        """
        Args:
            embedding_path (string): Path to  images.
            masks_path (string): path to masks.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.ground_truth_path = ground_truth_path
        self.device = device
        self.list_of_data = list_of_data
        self.embedding_path = embedding_path

    def __len__(self):
        return len(self.list_of_data)

    def __getitem__(self, idx):


      embedding = np.load(self.embedding_path + "/" + self.list_of_data[idx])
      embedding = torch.as_tensor(embedding, dtype=torch.float, device=self.device)


      #--------------------------------------------------
      case_id = self.list_of_data[idx].split("_")[1]
      slice_ = int(self.list_of_data[idx].split("_")[-1].split(".")[0])
      # "BraTS2021_00000"
      case_name = "BraTS2021_"+ case_id
      img =  nib.load(self.ground_truth_path + "/" + case_name + "/"+ case_name +"_seg.nii.gz")
      img = img.get_fdata()
      mask = img[:,:,slice_]

      mask[mask>0] = 1
      box = get_bounding_box(mask)
      # mask = squarify(mask)



      mask = transform.resize(
                      mask,
                      (256, 256),
                      order=0,
                      preserve_range=True,
                      mode="constant",
                  )


        # class_1 = np.zeros_like(mask)
        # class_2 = np.zeros_like(mask)
        # class_3 = np.zeros_like(mask)

        # class_1[np.where(mask == 1)] = 1
        # class_2[np.where(mask == 2)] = 1
        # class_3[np.where(mask == 4)] = 1

        # mask = mask = np.stack((class_1, class_2, class_3), axis = -1 )
      mask = torch.as_tensor(mask).to(self.device)
      mask = mask[None,...]


      box = get_bounding_box(mask)
      # box = [0,0,mask.shape[-2],mask.shape[-1]]
      box_np = np.array(box)
      sam_trans = ResizeLongestSide(sam_model.image_encoder.img_size)
      box = sam_trans.apply_boxes(box_np, (mask.shape[-2], mask.shape[-1]))
      box_torch = torch.as_tensor(box, dtype=torch.float, device=self.device)


      sample = {"embedding": embedding,
                  'mask': mask,
                  'box' : box_torch
                  }

      return sample


train = BraTSDataset(ground_truth_path, embedding_path, train_list)
val = BraTSDataset(ground_truth_path, embedding_path, val_list)



train_loader = DataLoader(train, batch_size= 8, shuffle=True, num_workers=0)
val_loader = DataLoader(val, batch_size= 4, shuffle=True, num_workers=0)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



hprams = {
    "num_epochs": 15,
    "best_loss": 1e10,
    "model_save_path": "/content",
    "lr":1e-5,
    "weight_decay":0,
    "device": device,
    "train_dataloader": train_loader,
    "val_dataloader": val_loader
    }

def eval(val_dataloader, sam_model, seg_loss):
  loss = 0
  for step_val, data in enumerate(val_dataloader):
    embedding, mask, box = data["embedding"], data["mask"], data["box"]

    with torch.no_grad():
      # embedding = sam_model.image_encoder(image)
      sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
          points=None,
          boxes=box,
          masks=None,
      )

      # predicted masks
      mask_predictions, _ = sam_model.mask_decoder(
      image_embeddings= embedding.to(device), # (B, 256, 64, 64)
      image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
      sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
      dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
      multimask_output=False,
      )

      loss += seg_loss(mask_predictions, mask.to(device))
  loss /= step_val+1
  return loss



def train(**kwargs):
    model_save_path = kwargs['model_save_path']
    num_epochs = kwargs['num_epochs']
    best_loss = kwargs['best_loss']
    lr = kwargs['lr']
    weight_decay = kwargs['weight_decay']
    device = kwargs['device']
    train_dataloader = kwargs['train_dataloader']
    # embedding_path = kwargs['embedding_path']
    val_dataloader = kwargs['val_dataloader']

    os.makedirs(model_save_path, exist_ok=True)

    sam_model.train()
    optimizer = torch.optim.Adam(sam_model.mask_decoder.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, factor=0.25,  patience=5)
    seg_loss = monai.losses.DiceLoss(sigmoid=True, squared_pred=True, reduction='mean')


    for epoch in range(num_epochs):
        epoch_loss = 0
        # train
        for step, data in enumerate(tqdm(train_dataloader)):
            mask, box, embedding =  data["mask"], data["box"], data["embedding"]
            embedding.to(device)

            with torch.no_grad():

              sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
                  points=None,
                  boxes=box,
                  masks=None,
              )

            # predicted masks
            mask_predictions, _ = sam_model.mask_decoder(
              image_embeddings= embedding.to(device), # (B, 256, 64, 64)
              image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
              sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
              dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
              multimask_output=False,
            )

            loss = seg_loss(mask_predictions, mask.to(device))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            # break
        epoch_loss /= step+1
        scheduler.step(epoch_loss)
        val_loss = None
        if val_dataloader is not None: val_loss = eval(val_dataloader, sam_model, seg_loss)
        print(f'EPOCH: {epoch}, Loss: {epoch_loss}, Val loss {val_loss}')
        # wandb.log({"epoch": epoch, "loss": loss})

        # save the latest model checkpoint
        # torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_'+str(epoch)+'.pth'))

        # save the best model
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_best.pth'))




train(**hprams)

In [ ]:
# 2d input 3d mask

print("No bounding box, flair_x")
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import Subset

from tqdm import tqdm
from torchsummary import summary
import monai
from segment_anything import SamPredictor, sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide
# import wandb
import nibabel as nib
from segment_anything.utils.transforms import ResizeLongestSide
import torch.nn.functional as F
from skimage import transform, io, segmentation


torch.manual_seed(2023)
np.random.seed(2023)

# change here
data_path = "/content/out"


def squarify(M,val = 0):
    (a,b)=M.shape
    if a>b:
        padding=((0,0),(0,a-b))
    else:
        padding=((0,b-a),(0,0))
    return np.pad(M,padding,mode='constant',constant_values=val)



def get_bounding_box(mask):
  x2 = mask.shape[0]
  y2 = mask.shape[1]

  return [0, 0, x2, y2]


def dice_loss(y_true, y_pred):
    """
    implementation for generalized dice score loss
    GDL = 1 - (2 * sum(intersection)/(sum(pred)+sum(true)))
    """

    # y_pred = tf.round(y_pred)
    intersection = torch.sum(y_true * y_pred)
    pred_sum = torch.sum( y_pred)
    true_sum = torch.sum( y_true)
    smooth = torch.ones_like(intersection)*1e-5
    return 1-((2*intersection+smooth)/(pred_sum+true_sum+smooth))


model_type = 'vit_h'
checkpoint = 'sam_vit_h_4b8939.pth'
sam_model = sam_model_registry[model_type](checkpoint=checkpoint).to('cuda')



embedding_path = "/content/BraTS_data/flair/z/embeddings"
ground_truth_path = "/content/out"

train_list = os.listdir(embedding_path)
train_list = train_list[:int(len(train_list)*0.7)]
val_list = train_list[int(len(train_list)*0.7):]

class BraTSDataset(Dataset):

    def __init__(self, ground_truth_path, embedding_path, list_of_data, device='cuda'):
        """
        Args:
            embedding_path (string): Path to  images.
            masks_path (string): path to masks.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.ground_truth_path = ground_truth_path
        self.device = device
        self.list_of_data = list_of_data
        self.embedding_path = embedding_path

    def __len__(self):
        return len(self.list_of_data)

    def __getitem__(self, idx):


      embedding = np.load(self.embedding_path + "/" + self.list_of_data[idx])
      embedding = torch.as_tensor(embedding, dtype=torch.float, device=self.device)


      #--------------------------------------------------
      case_id = self.list_of_data[idx].split("_")[1]
      slice_ = int(self.list_of_data[idx].split("_")[-1].split(".")[0])
      # "BraTS2021_00000"
      case_name = "BraTS2021_"+ case_id
      img =  nib.load(self.ground_truth_path + "/" + case_name + "/"+ case_name +"_seg.nii.gz")
      img = img.get_fdata()
      mask = img[slice_,:,:]

      mask[mask>0] = 1
      box = get_bounding_box(mask)
      # mask = squarify(mask)



      mask = transform.resize(
                      mask,
                      (256, 256),
                      order=0,
                      preserve_range=True,
                      mode="constant",
                  )


        # class_1 = np.zeros_like(mask)
        # class_2 = np.zeros_like(mask)
        # class_3 = np.zeros_like(mask)

        # class_1[np.where(mask == 1)] = 1
        # class_2[np.where(mask == 2)] = 1
        # class_3[np.where(mask == 4)] = 1

        # mask = mask = np.stack((class_1, class_2, class_3), axis = -1 )
      mask = torch.as_tensor(mask).to(self.device)
      mask = mask[None,...]


      box = get_bounding_box(mask)
      # box = [0,0,mask.shape[-2],mask.shape[-1]]
      box_np = np.array(box)
      sam_trans = ResizeLongestSide(sam_model.image_encoder.img_size)
      box = sam_trans.apply_boxes(box_np, (mask.shape[-2], mask.shape[-1]))
      box_torch = torch.as_tensor(box, dtype=torch.float, device=self.device)


      sample = {"embedding": embedding,
                  'mask': mask,
                  'box' : box_torch
                  }

      return sample


train = BraTSDataset(ground_truth_path, embedding_path, train_list)
val = BraTSDataset(ground_truth_path, embedding_path, val_list)



train_loader = DataLoader(train, batch_size= 8, shuffle=True, num_workers=0)
val_loader = DataLoader(val, batch_size= 4, shuffle=True, num_workers=0)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



hprams = {
    "num_epochs": 15,
    "best_loss": 1e10,
    "model_save_path": "/content",
    "lr":1e-5,
    "weight_decay":0,
    "device": device,
    "train_dataloader": train_loader,
    "val_dataloader": val_loader
    }

def eval(val_dataloader, sam_model, seg_loss):
  loss = 0
  for step_val, data in enumerate(val_dataloader):
    embedding, mask, box = data["embedding"], data["mask"], data["box"]

    with torch.no_grad():
      # embedding = sam_model.image_encoder(image)
      sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
          points=None,
          boxes=box,
          masks=None,
      )

      # predicted masks
      mask_predictions, _ = sam_model.mask_decoder(
      image_embeddings= embedding.to(device), # (B, 256, 64, 64)
      image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
      sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
      dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
      multimask_output=False,
      )

      loss += seg_loss(mask_predictions, mask.to(device))
  loss /= step_val+1
  return loss



def train(**kwargs):
    model_save_path = kwargs['model_save_path']
    num_epochs = kwargs['num_epochs']
    best_loss = kwargs['best_loss']
    lr = kwargs['lr']
    weight_decay = kwargs['weight_decay']
    device = kwargs['device']
    train_dataloader = kwargs['train_dataloader']
    # embedding_path = kwargs['embedding_path']
    val_dataloader = kwargs['val_dataloader']

    os.makedirs(model_save_path, exist_ok=True)

    sam_model.train()
    optimizer = torch.optim.Adam(sam_model.mask_decoder.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, factor=0.25,  patience=5)
    seg_loss = monai.losses.DiceLoss(sigmoid=True, squared_pred=True, reduction='mean')


    for epoch in range(num_epochs):
        epoch_loss = 0
        # train
        for step, data in enumerate(tqdm(train_dataloader)):
            mask, box, embedding =  data["mask"], data["box"], data["embedding"]
            embedding.to(device)

            with torch.no_grad():

              sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
                  points=None,
                  boxes=box,
                  masks=None,
              )

            # predicted masks
            mask_predictions, _ = sam_model.mask_decoder(
              image_embeddings= embedding.to(device), # (B, 256, 64, 64)
              image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
              sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
              dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
              multimask_output=False,
            )

            loss = seg_loss(mask_predictions, mask.to(device))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            # break
        epoch_loss /= step+1
        scheduler.step(epoch_loss)
        val_loss = None
        if val_dataloader is not None: val_loss = eval(val_dataloader, sam_model, seg_loss)
        print(f'EPOCH: {epoch}, Loss: {epoch_loss}, Val loss {val_loss}')
        # wandb.log({"epoch": epoch, "loss": loss})

        # save the latest model checkpoint
        # torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_'+str(epoch)+'.pth'))

        # save the best model
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_best.pth'))




train(**hprams)

In [ ]:
# 2d input 3d mask

print("No bounding box, flair_y")
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import Subset

from tqdm import tqdm
from torchsummary import summary
import monai
from segment_anything import SamPredictor, sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide
# import wandb
import nibabel as nib
from segment_anything.utils.transforms import ResizeLongestSide
import torch.nn.functional as F
from skimage import transform, io, segmentation


torch.manual_seed(2023)
np.random.seed(2023)

# change here
data_path = "/content/out"


def squarify(M,val = 0):
    (a,b)=M.shape
    if a>b:
        padding=((0,0),(0,a-b))
    else:
        padding=((0,b-a),(0,0))
    return np.pad(M,padding,mode='constant',constant_values=val)



def get_bounding_box(mask):
  x2 = mask.shape[0]
  y2 = mask.shape[1]

  return [0, 0, x2, y2]


def dice_loss(y_true, y_pred):
    """
    implementation for generalized dice score loss
    GDL = 1 - (2 * sum(intersection)/(sum(pred)+sum(true)))
    """

    # y_pred = tf.round(y_pred)
    intersection = torch.sum(y_true * y_pred)
    pred_sum = torch.sum( y_pred)
    true_sum = torch.sum( y_true)
    smooth = torch.ones_like(intersection)*1e-5
    return 1-((2*intersection+smooth)/(pred_sum+true_sum+smooth))


model_type = 'vit_h'
checkpoint = 'sam_vit_h_4b8939.pth'
sam_model = sam_model_registry[model_type](checkpoint=checkpoint).to('cuda')



embedding_path = "/content/BraTS_data/flair/z/embeddings"
ground_truth_path = "/content/out"

train_list = os.listdir(embedding_path)
train_list = train_list[:int(len(train_list)*0.7)]
val_list = train_list[int(len(train_list)*0.7):]

class BraTSDataset(Dataset):

    def __init__(self, ground_truth_path, embedding_path, list_of_data, device='cuda'):
        """
        Args:
            embedding_path (string): Path to  images.
            masks_path (string): path to masks.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.ground_truth_path = ground_truth_path
        self.device = device
        self.list_of_data = list_of_data
        self.embedding_path = embedding_path

    def __len__(self):
        return len(self.list_of_data)

    def __getitem__(self, idx):


      embedding = np.load(self.embedding_path + "/" + self.list_of_data[idx])
      embedding = torch.as_tensor(embedding, dtype=torch.float, device=self.device)


      #--------------------------------------------------
      case_id = self.list_of_data[idx].split("_")[1]
      slice_ = int(self.list_of_data[idx].split("_")[-1].split(".")[0])
      # "BraTS2021_00000"
      case_name = "BraTS2021_"+ case_id
      img =  nib.load(self.ground_truth_path + "/" + case_name + "/"+ case_name +"_seg.nii.gz")
      img = img.get_fdata()
      mask = img[:,slice_,:]

      mask[mask>0] = 1
      box = get_bounding_box(mask)
      # mask = squarify(mask)



      mask = transform.resize(
                      mask,
                      (256, 256),
                      order=0,
                      preserve_range=True,
                      mode="constant",
                  )


        # class_1 = np.zeros_like(mask)
        # class_2 = np.zeros_like(mask)
        # class_3 = np.zeros_like(mask)

        # class_1[np.where(mask == 1)] = 1
        # class_2[np.where(mask == 2)] = 1
        # class_3[np.where(mask == 4)] = 1

        # mask = mask = np.stack((class_1, class_2, class_3), axis = -1 )
      mask = torch.as_tensor(mask).to(self.device)
      mask = mask[None,...]


      box = get_bounding_box(mask)
      # box = [0,0,mask.shape[-2],mask.shape[-1]]
      box_np = np.array(box)
      sam_trans = ResizeLongestSide(sam_model.image_encoder.img_size)
      box = sam_trans.apply_boxes(box_np, (mask.shape[-2], mask.shape[-1]))
      box_torch = torch.as_tensor(box, dtype=torch.float, device=self.device)


      sample = {"embedding": embedding,
                  'mask': mask,
                  'box' : box_torch
                  }

      return sample


train = BraTSDataset(ground_truth_path, embedding_path, train_list)
val = BraTSDataset(ground_truth_path, embedding_path, val_list)



train_loader = DataLoader(train, batch_size= 8, shuffle=True, num_workers=0)
val_loader = DataLoader(val, batch_size= 4, shuffle=True, num_workers=0)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



hprams = {
    "num_epochs": 15,
    "best_loss": 1e10,
    "model_save_path": "/content",
    "lr":1e-5,
    "weight_decay":0,
    "device": device,
    "train_dataloader": train_loader,
    "val_dataloader": val_loader
    }

def eval(val_dataloader, sam_model, seg_loss):
  loss = 0
  for step_val, data in enumerate(val_dataloader):
    embedding, mask, box = data["embedding"], data["mask"], data["box"]

    with torch.no_grad():
      # embedding = sam_model.image_encoder(image)
      sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
          points=None,
          boxes=box,
          masks=None,
      )

      # predicted masks
      mask_predictions, _ = sam_model.mask_decoder(
      image_embeddings= embedding.to(device), # (B, 256, 64, 64)
      image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
      sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
      dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
      multimask_output=False,
      )

      loss += seg_loss(mask_predictions, mask.to(device))
  loss /= step_val+1
  return loss



def train(**kwargs):
    model_save_path = kwargs['model_save_path']
    num_epochs = kwargs['num_epochs']
    best_loss = kwargs['best_loss']
    lr = kwargs['lr']
    weight_decay = kwargs['weight_decay']
    device = kwargs['device']
    train_dataloader = kwargs['train_dataloader']
    # embedding_path = kwargs['embedding_path']
    val_dataloader = kwargs['val_dataloader']

    os.makedirs(model_save_path, exist_ok=True)

    sam_model.train()
    optimizer = torch.optim.Adam(sam_model.mask_decoder.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, factor=0.25,  patience=5)
    seg_loss = monai.losses.DiceLoss(sigmoid=True, squared_pred=True, reduction='mean')


    for epoch in range(num_epochs):
        epoch_loss = 0
        # train
        for step, data in enumerate(tqdm(train_dataloader)):
            mask, box, embedding =  data["mask"], data["box"], data["embedding"]
            embedding.to(device)

            with torch.no_grad():

              sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
                  points=None,
                  boxes=box,
                  masks=None,
              )

            # predicted masks
            mask_predictions, _ = sam_model.mask_decoder(
              image_embeddings= embedding.to(device), # (B, 256, 64, 64)
              image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
              sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
              dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
              multimask_output=False,
            )

            loss = seg_loss(mask_predictions, mask.to(device))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            # break
        epoch_loss /= step+1
        scheduler.step(epoch_loss)
        val_loss = None
        if val_dataloader is not None: val_loss = eval(val_dataloader, sam_model, seg_loss)
        print(f'EPOCH: {epoch}, Loss: {epoch_loss}, Val loss {val_loss}')
        # wandb.log({"epoch": epoch, "loss": loss})

        # save the latest model checkpoint
        # torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_'+str(epoch)+'.pth'))

        # save the best model
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_best.pth'))




train(**hprams)

## Pipeline for 3D embeddings

In [ ]:
# 3d embedding saved

print("")
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import Subset

from tqdm import tqdm
from torchsummary import summary
import monai
from segment_anything import SamPredictor, sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide
# import wandb
import nibabel as nib
from segment_anything.utils.transforms import ResizeLongestSide
import torch.nn.functional as F
from skimage import transform, io, segmentation
import random



torch.manual_seed(2023)
np.random.seed(2023)

# change here
data_path = "/content/out"


def squarify(M,val = 0):
    (a,b)=M.shape
    if a>b:
        padding=((0,0),(0,a-b))
    else:
        padding=((0,b-a),(0,0))
    return np.pad(M,padding,mode='constant',constant_values=val)



def get_bounding_box(mask):
  x2 = mask.shape[-2]
  y2 = mask.shape[-1]

  return [0, 0, x2, y2]


def dice_loss(y_true, y_pred):
    """
    implementation for generalized dice score loss
    GDL = 1 - (2 * sum(intersection)/(sum(pred)+sum(true)))
    """

    # y_pred = tf.round(y_pred)
    intersection = torch.sum(y_true * y_pred)
    pred_sum = torch.sum( y_pred)
    true_sum = torch.sum( y_true)
    smooth = torch.ones_like(intersection)*1e-5
    return 1-((2*intersection+smooth)/(pred_sum+true_sum+smooth))


model_type = 'vit_h'
checkpoint = 'sam_vit_h_4b8939.pth'
sam_model = sam_model_registry[model_type](checkpoint=checkpoint).to('cuda')



embedding_path = "/content/BraTS_data/flair"
ground_truth_path = "/content/out"

train_list = os.listdir(embedding_path)
train_list = train_list[:int(len(train_list)*0.7)]
val_list = train_list[int(len(train_list)*0.7):]

class BraTSDataset(Dataset):

    def __init__(self, ground_truth_path, embedding_path, list_of_data, dimention, device='cuda'):
        """
        Args:
            embedding_path (string): Path to  images.
            masks_path (string): path to masks.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.ground_truth_path = ground_truth_path
        self.device = device
        self.list_of_data = list_of_data
        self.embedding_path = embedding_path
        self.dimention = dimention

    def __len__(self):
      return len(self.list_of_data)*240

    def __getitem__(self, idx):
      case_num = idx % len(os.listdir(self.ground_truth_path))
      dimention = random.choice(["x", "y", "z"])
      case_name = sorted(os.listdir(self.ground_truth_path))[case_num]

      img =  nib.load(self.ground_truth_path + "/" + case_name + "/"+ case_name +"_seg.nii.gz")
      img = img.get_fdata()

      if dimention == "z":
        num_slices = img.shape[-1]
        slice_num = random.randint(0, num_slices)
        mask = img[:,:,slice_num]
      if dimention == "y":
        num_slices = img.shape[1]
        slice_num = random.randint(0, num_slices)
        mask = img[:,slice_num,:]
      if dimention == "x":
        num_slices = img.shape[0]
        slice_num = random.randint(0, num_slices)
        mask = img[slice_num,:,:]


      3d_embedding = np.load(self.embedding_path + "/" + dimention + "/embeddings/" + case_name + ".npy")
      embedding = 3d_embedding[slice_num,:,:,:]
      embedding = torch.as_tensor(embedding, dtype=torch.float, device=self.device)

      #--------------------------------------------------


      mask[mask>0] = 1
      if self.dimention != "z":
        mask = squarify(mask)

      mask = transform.resize(
                      mask,
                      (256, 256),
                      order=0,
                      preserve_range=True,
                      mode="constant",
                  )


        # class_1 = np.zeros_like(mask)
        # class_2 = np.zeros_like(mask)
        # class_3 = np.zeros_like(mask)

        # class_1[np.where(mask == 1)] = 1
        # class_2[np.where(mask == 2)] = 1
        # class_3[np.where(mask == 4)] = 1

        # mask = mask = np.stack((class_1, class_2, class_3), axis = -1 )
      mask = torch.as_tensor(mask).to(self.device)
      mask = mask[None,...]


      box = get_bounding_box(mask)
      # box = [0,0,mask.shape[-2],mask.shape[-1]]
      box_np = np.array(box)
      sam_trans = ResizeLongestSide(sam_model.image_encoder.img_size)
      box = sam_trans.apply_boxes(box_np, (mask.shape[-2], mask.shape[-1]))
      box_torch = torch.as_tensor(box, dtype=torch.float, device=self.device)


      sample = {"embedding": embedding,
                  'mask': mask,
                  'box' : box_torch
                  }

      return sample


train = BraTSDataset(ground_truth_path, embedding_path, train_list)
val = BraTSDataset(ground_truth_path, embedding_path, val_list)



train_loader = DataLoader(train, batch_size= 8, shuffle=True, num_workers=0)
val_loader = DataLoader(val, batch_size= 4, shuffle=True, num_workers=0)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



hprams = {
    "num_epochs": 15,
    "best_loss": 1e10,
    "model_save_path": "/content",
    "lr":1e-5,
    "weight_decay":0,
    "device": device,
    "train_dataloader": train_loader,
    "val_dataloader": val_loader
    }

def eval(val_dataloader, sam_model, seg_loss):
  loss = 0
  for step_val, data in enumerate(val_dataloader):
    embedding, mask, box = data["embedding"], data["mask"], data["box"]

    with torch.no_grad():
      # embedding = sam_model.image_encoder(image)
      sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
          points=None,
          boxes=box,
          masks=None,
      )

      # predicted masks
      mask_predictions, _ = sam_model.mask_decoder(
      image_embeddings= embedding.to(device), # (B, 256, 64, 64)
      image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
      sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
      dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
      multimask_output=False,
      )

      loss += seg_loss(mask_predictions, mask.to(device))
  loss /= step_val+1
  return loss



def train(**kwargs):
    model_save_path = kwargs['model_save_path']
    num_epochs = kwargs['num_epochs']
    best_loss = kwargs['best_loss']
    lr = kwargs['lr']
    weight_decay = kwargs['weight_decay']
    device = kwargs['device']
    train_dataloader = kwargs['train_dataloader']
    # embedding_path = kwargs['embedding_path']
    val_dataloader = kwargs['val_dataloader']

    os.makedirs(model_save_path, exist_ok=True)

    sam_model.train()
    optimizer = torch.optim.Adam(sam_model.mask_decoder.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, factor=0.25,  patience=5)
    seg_loss = monai.losses.DiceLoss(sigmoid=True, squared_pred=True, reduction='mean')


    for epoch in range(num_epochs):
        epoch_loss = 0
        # train
        for step, data in enumerate(tqdm(train_dataloader)):
            mask, box, embedding =  data["mask"], data["box"], data["embedding"]
            embedding.to(device)

            with torch.no_grad():

              sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
                  points=None,
                  boxes=box,
                  masks=None,
              )

            # predicted masks
            mask_predictions, _ = sam_model.mask_decoder(
              image_embeddings= embedding.to(device), # (B, 256, 64, 64)
              image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
              sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
              dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
              multimask_output=False,
            )

            loss = seg_loss(mask_predictions, mask.to(device))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            # break
        epoch_loss /= step+1
        scheduler.step(epoch_loss)
        val_loss = None
        if val_dataloader is not None: val_loss = eval(val_dataloader, sam_model, seg_loss)
        print(f'EPOCH: {epoch}, Loss: {epoch_loss}, Val loss {val_loss}')
        # wandb.log({"epoch": epoch, "loss": loss})

        # save the latest model checkpoint
        # torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_'+str(epoch)+'.pth'))

        # save the best model
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_best.pth'))




train(**hprams)

## Pipeline for saved 2d input and 3d mask bounding box

In [ ]:
# 2d input 3d mask



print("Bounding box, flair_z")
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import Subset

from tqdm import tqdm
from torchsummary import summary
import monai
from segment_anything import SamPredictor, sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide
# import wandb
import nibabel as nib
from segment_anything.utils.transforms import ResizeLongestSide
import torch.nn.functional as F
from skimage import transform, io, segmentation
import random


torch.manual_seed(2023)
np.random.seed(2023)

# change here
data_path = "/content/out"


def squarify(M,val = 0):
    (a,b)=M.shape
    if a>b:
        padding=((0,0),(0,a-b))
    else:
        padding=((0,b-a),(0,0))
    return np.pad(M,padding,mode='constant',constant_values=val)



def get_bounding_box(mask):

  y_indices, x_indices = np.where(mask > 0)
  if len(y_indices) == 0:
    return [0,0,0,0]
  x_min, x_max = np.min(x_indices), np.max(x_indices)
  y_min, y_max = np.min(y_indices), np.max(y_indices)
  # add perturbation to bounding box coordinates
  H, W = mask.shape
  x_min = max(0, x_min - np.random.randint(0, 20))
  x_max = min(W, x_max + np.random.randint(0, 20))
  y_min = max(0, y_min - np.random.randint(0, 20))
  y_max = min(H, y_max + np.random.randint(0, 20))
  # bboxes = np.array([x_min, y_min, x_max, y_max])

  return [x_min, y_min, x_max, y_max]


def get_points(mask):
  mask= mask.cpu().numpy()
  mask = np.squeeze(mask, axis = 0)
  x = np.where(mask>0)
  if len(x[0]) == 0:
    return (0,0)
  all_points = list(zip(x[0], x[1]))
  return random.choices(all_points)


def dice_loss(y_true, y_pred):
    """
    implementation for generalized dice score loss
    GDL = 1 - (2 * sum(intersection)/(sum(pred)+sum(true)))
    """

    # y_pred = tf.round(y_pred)
    intersection = torch.sum(y_true * y_pred)
    pred_sum = torch.sum( y_pred)
    true_sum = torch.sum( y_true)
    smooth = torch.ones_like(intersection)*1e-5
    return 1-((2*intersection+smooth)/(pred_sum+true_sum+smooth))


model_type = 'vit_h'
checkpoint = 'sam_vit_h_4b8939.pth'
sam_model = sam_model_registry[model_type](checkpoint=checkpoint).to('cuda')



embedding_path = "/content/BraTS_data/flair/z/embeddings"
ground_truth_path = "/content/out"

train_list = os.listdir(embedding_path)
train_list = train_list[:int(len(train_list)*0.7)]
val_list = train_list[int(len(train_list)*0.7):]

class BraTSDataset(Dataset):

    def __init__(self, ground_truth_path, embedding_path, list_of_data, device='cuda'):
        """
        Args:
            embedding_path (string): Path to  images.
            masks_path (string): path to masks.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.ground_truth_path = ground_truth_path
        self.device = device
        self.list_of_data = list_of_data
        self.embedding_path = embedding_path

    def __len__(self):
        return len(self.list_of_data)

    def __getitem__(self, idx):


      embedding = np.load(self.embedding_path + "/" + self.list_of_data[idx])
      embedding = torch.as_tensor(embedding, dtype=torch.float, device=self.device)


      #--------------------------------------------------
      case_id = self.list_of_data[idx].split("_")[1]
      slice_ = int(self.list_of_data[idx].split("_")[-1].split(".")[0])
      # "BraTS2021_00000"
      case_name = "BraTS2021_"+ case_id
      img =  nib.load(self.ground_truth_path + "/" + case_name + "/"+ case_name +"_seg.nii.gz")
      img = img.get_fdata()
      mask = img[:,:,slice_]

      mask[mask>0] = 1
      # mask = squarify(mask)



      mask = transform.resize(
                      mask,
                      (256, 256),
                      order=0,
                      preserve_range=True,
                      mode="constant",
                  )

        # class_1 = np.zeros_like(mask)
        # class_2 = np.zeros_like(mask)
        # class_3 = np.zeros_like(mask)

        # class_1[np.where(mask == 1)] = 1
        # class_2[np.where(mask == 2)] = 1
        # class_3[np.where(mask == 4)] = 1

        # mask = mask = np.stack((class_1, class_2, class_3), axis = -1 )


      box = get_bounding_box(mask)
      # box = [0,0,mask.shape[-2],mask.shape[-1]]
      box_np = np.array(box)
      sam_trans = ResizeLongestSide(sam_model.image_encoder.img_size)
      box = sam_trans.apply_boxes(box_np, (mask.shape[-2], mask.shape[-1]))
      box_torch = torch.as_tensor(box, dtype=torch.float, device=self.device)

      mask = torch.as_tensor(mask).to(self.device)
      mask = mask[None,...]


      sample = {"embedding": embedding,
                  'mask': mask,
                  'box' : box_torch
                  }

      return sample


train = BraTSDataset(ground_truth_path, embedding_path, train_list)
val = BraTSDataset(ground_truth_path, embedding_path, val_list)



train_loader = DataLoader(train, batch_size= 2, shuffle=True, num_workers=0)
val_loader = DataLoader(val, batch_size= 1, shuffle=True, num_workers=0)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



hprams = {
    "num_epochs": 10,
    "best_loss": 1e10,
    "model_save_path": "/content",
    "lr":1e-4,
    "weight_decay":0,
    "device": device,
    "train_dataloader": train_loader,
    "val_dataloader": val_loader
    }

def eval(val_dataloader, sam_model, seg_loss):
  loss = 0
  for step_val, data in enumerate(val_dataloader):
    embedding, mask, box = data["embedding"], data["mask"], data["box"]

    with torch.no_grad():
      # embedding = sam_model.image_encoder(image)
      sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
          points=None,
          boxes=box,
          masks=None,
      )

      # predicted masks
      mask_predictions, _ = sam_model.mask_decoder(
      image_embeddings= embedding.to(device), # (B, 256, 64, 64)
      image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
      sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
      dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
      multimask_output=False,
      )

      loss += seg_loss(mask_predictions, mask.to(device))
  loss /= step_val+1
  return loss



def train(**kwargs):
    model_save_path = kwargs['model_save_path']
    num_epochs = kwargs['num_epochs']
    best_loss = kwargs['best_loss']
    lr = kwargs['lr']
    weight_decay = kwargs['weight_decay']
    device = kwargs['device']
    train_dataloader = kwargs['train_dataloader']
    # embedding_path = kwargs['embedding_path']
    val_dataloader = kwargs['val_dataloader']

    os.makedirs(model_save_path, exist_ok=True)

    sam_model.train()
    optimizer = torch.optim.Adam(sam_model.mask_decoder.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, factor=0.25,  patience=3)
    seg_loss = monai.losses.DiceLoss(sigmoid=True, squared_pred=True, reduction='mean')


    for epoch in range(num_epochs):
        epoch_loss = 0
        # train
        for step, data in enumerate(tqdm(train_dataloader)):
            mask, box, embedding =  data["mask"], data["box"], data["embedding"]

            embedding.to(device)

            with torch.no_grad():

              sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
                  points=None,
                  boxes=box,
                  masks=None,
              )

            # predicted masks
            mask_predictions, _ = sam_model.mask_decoder(
              image_embeddings= embedding.to(device), # (B, 256, 64, 64)
              image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
              sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
              dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
              multimask_output=False,
            )

            loss = seg_loss(mask_predictions, mask.to(device))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            # break
        epoch_loss /= step+1
        scheduler.step(epoch_loss)
        val_loss = None
        if val_dataloader is not None: val_loss = eval(val_dataloader, sam_model, seg_loss)
        print(f'EPOCH: {epoch}, Loss: {epoch_loss}, Val loss {val_loss}')
        # wandb.log({"epoch": epoch, "loss": loss})

        # save the latest model checkpoint
        # torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_'+str(epoch)+'.pth'))

        # save the best model
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_best.pth'))




train(**hprams)

## 3D embedding for all modalities

In [ ]:
# 3d embedding saved  #binary

print("")
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import Subset

from tqdm import tqdm
from torchsummary import summary
import monai
from segment_anything import SamPredictor, sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide
# import wandb
import nibabel as nib
from segment_anything.utils.transforms import ResizeLongestSide
import torch.nn.functional as F
from skimage import transform, io, segmentation
import random



torch.manual_seed(2023)
np.random.seed(2023)

# change here
data_path = "/content/out"


def squarify(M,val = 0):
    (a,b)=M.shape
    if a>b:
        padding=((0,0),(0,a-b))
    else:
        padding=((0,b-a),(0,0))
    return np.pad(M,padding,mode='constant',constant_values=val)



def get_bounding_box(mask):

  y_indices, x_indices = np.where(mask > 0)
  if len(y_indices) == 0:
    return [0,0,0,0]
  x_min, x_max = np.min(x_indices), np.max(x_indices)
  y_min, y_max = np.min(y_indices), np.max(y_indices)
  # add perturbation to bounding box coordinates
  H, W = mask.shape
  x_min = max(0, x_min - np.random.randint(0, 20))
  x_max = min(W, x_max + np.random.randint(0, 20))
  y_min = max(0, y_min - np.random.randint(0, 20))
  y_max = min(H, y_max + np.random.randint(0, 20))
  # bboxes = np.array([x_min, y_min, x_max, y_max])

  return [x_min, y_min, x_max, y_max]


def get_cases_list(input_base, masks_path):
  images = []
  for i in os.listdir(masks_path):
  # for i in  ["BraTS-SSA-00002-000"]:
    img =  nib.load(masks_path + "/" + i + "/"+ i +"-seg.nii.gz")
    img = img.get_fdata()
    sclices_z = img.shape[2]
    sclices_y = img.shape[1]
    sclices_x = img.shape[0]
    for z in range(sclices_z):
      if img[:,:,z].max() != 0:
        images.append((i ,z, "flair/z"))
        images.append((i ,z, "t1/z"))
        images.append((i ,z, "t2/z"))
        images.append((i ,z, "t1ce/z"))

    for y in range(sclices_y):
      if img[:,y,:].max() != 0:
        images.append((i ,y, "flair/y"))
        images.append((i ,y, "t1/y"))
        images.append((i ,y, "t2/y"))
        images.append((i ,y, "t1ce/y"))


    for x in range(sclices_x):
      if img[x,:,:].max() != 0:
        images.append(( i ,x, "flair/x"))
        images.append(( i ,x, "t1/x"))
        images.append((i ,x, "t2/x"))
        images.append(( i ,x, "t1ce/x"))

    return images


def dice_loss(y_true, y_pred):
    """
    implementation for generalized dice score loss
    GDL = 1 - (2 * sum(intersection)/(sum(pred)+sum(true)))
    """

    # y_pred = tf.round(y_pred)
    intersection = torch.sum(y_true * y_pred)
    pred_sum = torch.sum( y_pred)
    true_sum = torch.sum( y_true)
    smooth = torch.ones_like(intersection)*1e-5
    return 1-((2*intersection+smooth)/(pred_sum+true_sum+smooth))


model_type = 'vit_h'
checkpoint = 'sam_vit_h_4b8939.pth'
sam_model = sam_model_registry[model_type](checkpoint=checkpoint).to('cuda')



embedding_path = "/content/BraTS_data"
ground_truth_path = "/content/drive/MyDrive/SSA"

data = get_cases_list(embedding_path, ground_truth_path)
print(len(data))
train_list = data[:int(len(data)*0.7)]
val_list = data[int(len(data)*0.7):]

class BraTSDataset(Dataset):

    def __init__(self, ground_truth_path, embedding_path, list_of_data, device='cuda'):
        """
        Args:
            embedding_path (string): Path to  images.
            masks_path (string): path to masks.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.ground_truth_path = ground_truth_path
        self.device = device
        self.list_of_data = list_of_data
        self.embedding_path = embedding_path

    def __len__(self):
      return len(self.list_of_data)

    def __getitem__(self, idx):
      case_name, slic, d = self.list_of_data[idx]

      img =  nib.load(self.ground_truth_path + "/" + case_name.split(".")[0] + "/"+ case_name.split(".")[0] +"-seg.nii.gz")
      img = img.get_fdata()

      dim =  d.split("/")[-1]
      if dim == "x":
        mask = img[slic,:,:]
      elif dim == "y":
        mask = img[:,slic,:]
      elif dim == "z":
        mask = img[:,:,slic]


      d_embedding = np.load(self.embedding_path + "/" + d + "/embeddings/" + case_name + ".npy")
      embedding = d_embedding[:,:,:, slic]
      embedding = torch.as_tensor(embedding, dtype=torch.float, device=self.device)

      #--------------------------------------------------
      mask[mask>0] = 1
      mask = transform.resize(
                      mask,
                      (256, 256),
                      order=0,
                      preserve_range=True,
                      mode="constant",
                  )

      box = get_bounding_box(mask)
      box_np = np.array(box)
      sam_trans = ResizeLongestSide(sam_model.image_encoder.img_size)
      box = sam_trans.apply_boxes(box_np, (mask.shape[-2], mask.shape[-1]))
      box_torch = torch.as_tensor(box, dtype=torch.float, device=self.device)

      # class_1 = np.zeros_like(mask)
      # class_2 = np.zeros_like(mask)
      # class_3 = np.zeros_like(mask)

      # class_1[np.where(mask == 1)] = 1
      # class_2[np.where(mask == 2)] = 1
      # class_3[np.where(mask == 3)] = 1

      # mask = mask = np.stack((class_1, class_2, class_3), axis = -1 )


      mask = torch.as_tensor(mask).to(self.device)
      mask = mask[None,...]

      sample = {"embedding": embedding,
                  'mask': mask,
                  'box' : box_torch
                  }

      return sample


train = BraTSDataset(ground_truth_path, embedding_path, train_list)
val = BraTSDataset(ground_truth_path, embedding_path, val_list)


train_loader = DataLoader(train, batch_size= 8, shuffle=True, num_workers=0)
val_loader = DataLoader(val, batch_size= 4, shuffle=True, num_workers=0)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



hprams = {
    "num_epochs": 5,
    "best_loss": 1e10,
    "model_save_path": "/content",
    "lr":1e-5,
    "weight_decay":0,
    "device": device,
    "train_dataloader": train_loader,
    "val_dataloader": val_loader
    }

def eval(val_dataloader, sam_model, seg_loss):
  loss = 0
  for step_val, data in enumerate(val_dataloader):
    embedding, mask, box = data["embedding"], data["mask"], data["box"]

    with torch.no_grad():
      # embedding = sam_model.image_encoder(image)
      sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
          points=None,
          boxes=box,
          masks=None,
      )

      # predicted masks
      mask_predictions, _ = sam_model.mask_decoder(
      image_embeddings= embedding.to(device), # (B, 256, 64, 64)
      image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
      sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
      dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
      multimask_output=False,
      )

      loss += seg_loss(mask_predictions, mask.to(device))
  loss /= step_val+1
  return loss



def train(**kwargs):
    model_save_path = kwargs['model_save_path']
    num_epochs = kwargs['num_epochs']
    best_loss = kwargs['best_loss']
    lr = kwargs['lr']
    weight_decay = kwargs['weight_decay']
    device = kwargs['device']
    train_dataloader = kwargs['train_dataloader']
    # embedding_path = kwargs['embedding_path']
    val_dataloader = kwargs['val_dataloader']

    os.makedirs(model_save_path, exist_ok=True)

    sam_model.train()
    optimizer = torch.optim.Adam(sam_model.mask_decoder.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, factor=0.25,  patience=5)
    seg_loss = monai.losses.DiceLoss(sigmoid=True, squared_pred=True, reduction='mean')


    for epoch in range(num_epochs):
        epoch_loss = 0
        # train
        for step, data in enumerate(tqdm(train_dataloader)):
            mask, box, embedding =  data["mask"], data["box"], data["embedding"]
            embedding.to(device)

            with torch.no_grad():

              sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
                  points=None,
                  boxes=box,
                  masks=None,
              )

            # predicted masks
            mask_predictions, _ = sam_model.mask_decoder(
              image_embeddings= embedding.to(device), # (B, 256, 64, 64)
              image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
              sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
              dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
              multimask_output=False,
            )

            loss = seg_loss(mask_predictions, mask.to(device))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            # break
        epoch_loss /= step+1
        scheduler.step(epoch_loss)
        val_loss = None
        if val_dataloader is not None: val_loss = eval(val_dataloader, sam_model, seg_loss)
        print(f'EPOCH: {epoch}, Loss: {epoch_loss}, Val loss {val_loss}')
        # wandb.log({"epoch": epoch, "loss": loss})

        # save the latest model checkpoint
        # torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_'+str(epoch)+'.pth'))

        # save the best model
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_best.pth'))




train(**hprams)

In [ ]:
# 3d embedding saved  # multiclass

print("")
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import Subset

from tqdm import tqdm
from torchsummary import summary
import monai
from segment_anything import SamPredictor, sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide
# import wandb
import nibabel as nib
from segment_anything.utils.transforms import ResizeLongestSide
import torch.nn.functional as F
from skimage import transform, io, segmentation
import random



torch.manual_seed(2023)
np.random.seed(2023)

# change here
data_path = "/content/out"


def squarify(M,val = 0):
    (a,b)=M.shape
    if a>b:
        padding=((0,0),(0,a-b))
    else:
        padding=((0,b-a),(0,0))
    return np.pad(M,padding,mode='constant',constant_values=val)



def get_bounding_box(mask):

  y_indices, x_indices = np.where(mask > 0)
  if len(y_indices) == 0:
    return [0,0,0,0]
  x_min, x_max = np.min(x_indices), np.max(x_indices)
  y_min, y_max = np.min(y_indices), np.max(y_indices)
  # add perturbation to bounding box coordinates
  H, W = mask.shape
  x_min = max(0, x_min - np.random.randint(0, 20))
  x_max = min(W, x_max + np.random.randint(0, 20))
  y_min = max(0, y_min - np.random.randint(0, 20))
  y_max = min(H, y_max + np.random.randint(0, 20))
  # bboxes = np.array([x_min, y_min, x_max, y_max])

  return [x_min, y_min, x_max, y_max]


def get_cases_list(input_base, masks_path):
  images = []
  for i in os.listdir(masks_path):
  # for i in  ["BraTS-SSA-00002-000"]:
    img =  nib.load(masks_path + "/" + i + "/"+ i +"-seg.nii.gz")
    img = img.get_fdata()
    sclices_z = img.shape[2]
    sclices_y = img.shape[1]
    sclices_x = img.shape[0]
    for z in range(sclices_z):
      if img[:,:,z].max() != 0:
        images.append((i ,z, "flair/z"))
        images.append((i ,z, "t1/z"))
        images.append((i ,z, "t2/z"))
        images.append((i ,z, "t1ce/z"))

    for y in range(sclices_y):
      if img[:,y,:].max() != 0:
        images.append((i ,y, "flair/y"))
        images.append((i ,y, "t1/y"))
        images.append((i ,y, "t2/y"))
        images.append((i ,y, "t1ce/y"))


    for x in range(sclices_x):
      if img[x,:,:].max() != 0:
        images.append(( i ,x, "flair/x"))
        images.append(( i ,x, "t1/x"))
        images.append((i ,x, "t2/x"))
        images.append(( i ,x, "t1ce/x"))

    return images


def dice_loss(y_true, y_pred):
    """
    implementation for generalized dice score loss
    GDL = 1 - (2 * sum(intersection)/(sum(pred)+sum(true)))
    """

    # y_pred = tf.round(y_pred)
    intersection = torch.sum(y_true * y_pred)
    pred_sum = torch.sum( y_pred)
    true_sum = torch.sum( y_true)
    smooth = torch.ones_like(intersection)*1e-5
    return 1-((2*intersection+smooth)/(pred_sum+true_sum+smooth))


model_type = 'vit_h'
checkpoint = 'sam_vit_h_4b8939.pth'
sam_model = sam_model_registry[model_type](checkpoint=checkpoint).to('cuda')



embedding_path = "/content/BraTS_data"
ground_truth_path = "/content/drive/MyDrive/SSA"

data = get_cases_list(embedding_path, ground_truth_path)
print(len(data))
train_list = data[:int(len(data)*0.7)]
val_list = data[int(len(data)*0.7):]

class BraTSDataset(Dataset):

    def __init__(self, ground_truth_path, embedding_path, list_of_data, device='cuda'):
        """
        Args:
            embedding_path (string): Path to  images.
            masks_path (string): path to masks.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.ground_truth_path = ground_truth_path
        self.device = device
        self.list_of_data = list_of_data
        self.embedding_path = embedding_path

    def __len__(self):
      return len(self.list_of_data)

    def __getitem__(self, idx):
      case_name, slic, d = self.list_of_data[idx]

      img =  nib.load(self.ground_truth_path + "/" + case_name.split(".")[0] + "/"+ case_name.split(".")[0] +"-seg.nii.gz")
      img = img.get_fdata()

      dim =  d.split("/")[-1]
      if dim == "x":
        mask = img[slic,:,:]
      elif dim == "y":
        mask = img[:,slic,:]
      elif dim == "z":
        mask = img[:,:,slic]


      d_embedding = np.load(self.embedding_path + "/" + d + "/embeddings/" + case_name + ".npy")
      embedding = d_embedding[:,:,:, slic]
      embedding = torch.as_tensor(embedding, dtype=torch.float, device=self.device)

      #--------------------------------------------------
      # mask[mask>0] = 1
      mask = transform.resize(
                      mask,
                      (256, 256),
                      order=0,
                      preserve_range=True,
                      mode="constant",
                  )

      box = get_bounding_box(mask)
      box_np = np.array(box)
      sam_trans = ResizeLongestSide(sam_model.image_encoder.img_size)
      box = sam_trans.apply_boxes(box_np, (mask.shape[-2], mask.shape[-1]))
      box_torch = torch.as_tensor(box, dtype=torch.float, device=self.device)

      class_1 = np.zeros_like(mask)
      class_2 = np.zeros_like(mask)
      class_3 = np.zeros_like(mask)

      class_1[np.where(mask == 1)] = 1
      class_2[np.where(mask == 2)] = 1
      class_3[np.where(mask == 3)] = 1

      mask = mask = np.stack((class_1, class_2, class_3), axis = 0)


      mask = torch.as_tensor(mask).to(self.device)
      # mask = mask[None,...]

      sample = {"embedding": embedding,
                  'mask': mask,
                  'box' : box_torch
                  }

      return sample


train = BraTSDataset(ground_truth_path, embedding_path, train_list)
val = BraTSDataset(ground_truth_path, embedding_path, val_list)


train_loader = DataLoader(train, batch_size= 8, shuffle=True, num_workers=0)
val_loader = DataLoader(val, batch_size= 4, shuffle=True, num_workers=0)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



hprams = {
    "num_epochs": 5,
    "best_loss": 1e10,
    "model_save_path": "/content",
    "lr":1e-5,
    "weight_decay":0,
    "device": device,
    "train_dataloader": train_loader,
    "val_dataloader": val_loader
    }

def eval(val_dataloader, sam_model, seg_loss):
  loss = 0
  for step_val, data in enumerate(val_dataloader):
    embedding, mask, box = data["embedding"], data["mask"], data["box"]

    with torch.no_grad():
      # embedding = sam_model.image_encoder(image)
      sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
          points=None,
          boxes=box,
          masks=None,
      )

      # predicted masks
      mask_predictions, _ = sam_model.mask_decoder(
      image_embeddings= embedding.to(device), # (B, 256, 64, 64)
      image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
      sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
      dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
      multimask_output=True,
      )

      loss += seg_loss(mask_predictions, mask.to(device))
  loss /= step_val+1
  return loss



def train(**kwargs):
    model_save_path = kwargs['model_save_path']
    num_epochs = kwargs['num_epochs']
    best_loss = kwargs['best_loss']
    lr = kwargs['lr']
    weight_decay = kwargs['weight_decay']
    device = kwargs['device']
    train_dataloader = kwargs['train_dataloader']
    # embedding_path = kwargs['embedding_path']
    val_dataloader = kwargs['val_dataloader']

    os.makedirs(model_save_path, exist_ok=True)

    sam_model.train()
    optimizer = torch.optim.Adam(sam_model.mask_decoder.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, factor=0.25,  patience=5)
    seg_loss = monai.losses.DiceLoss(sigmoid=True, squared_pred=True, reduction='mean')


    for epoch in range(num_epochs):
        epoch_loss = 0
        # train
        for step, data in enumerate(tqdm(train_dataloader)):
            mask, box, embedding =  data["mask"], data["box"], data["embedding"]
            embedding.to(device)

            with torch.no_grad():

              sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
                  points=None,
                  boxes=box,
                  masks=None,
              )

            # predicted masks
            mask_predictions, _ = sam_model.mask_decoder(
              image_embeddings= embedding.to(device), # (B, 256, 64, 64)
              image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
              sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
              dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
              multimask_output=True,
            )

            loss = seg_loss(mask_predictions, mask.to(device))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            # break
        epoch_loss /= step+1
        scheduler.step(epoch_loss)
        val_loss = None
        if val_dataloader is not None: val_loss = eval(val_dataloader, sam_model, seg_loss)
        print(f'EPOCH: {epoch}, Loss: {epoch_loss}, Val loss {val_loss}')
        # wandb.log({"epoch": epoch, "loss": loss})

        # save the latest model checkpoint
        # torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_'+str(epoch)+'.pth'))

        # save the best model
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_best.pth'))




train(**hprams)

## Optimized 3d for all modalities

In [ ]:
# 3d embedding saved  #multi class

print("")
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import Subset

from tqdm import tqdm
from torchsummary import summary
import monai
from segment_anything import SamPredictor, sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide
# import wandb
import nibabel as nib
from segment_anything.utils.transforms import ResizeLongestSide
import torch.nn.functional as F
from skimage import transform, io, segmentation
import random
from monai.data import ThreadDataLoader, CacheDataset



torch.manual_seed(2023)
np.random.seed(2023)



def squarify(M,val = 0):
    (a,b)=M.shape
    if a>b:
        padding=((0,0),(0,a-b))
    else:
        padding=((0,b-a),(0,0))
    return np.pad(M,padding,mode='constant',constant_values=val)



def get_bounding_box(mask):

  y_indices, x_indices = np.where(mask > 0)
  if len(y_indices) == 0:
    return [0,0,0,0]
  x_min, x_max = np.min(x_indices), np.max(x_indices)
  y_min, y_max = np.min(y_indices), np.max(y_indices)
  # add perturbation to bounding box coordinates
  H, W = mask.shape
  x_min = max(0, x_min - np.random.randint(0, 20))
  x_max = min(W, x_max + np.random.randint(0, 20))
  y_min = max(0, y_min - np.random.randint(0, 20))
  y_max = min(H, y_max + np.random.randint(0, 20))
  # bboxes = np.array([x_min, y_min, x_max, y_max])

  return [x_min, y_min, x_max, y_max]


def get_cases_list(input_base, masks_path):
  images = []
  for i in os.listdir(masks_path):
  # for i in  ["BraTS-SSA-00002-000"]:
    img =  nib.load(masks_path + "/" + i + "/"+ i +"-seg.nii.gz")
    img = img.get_fdata()
    sclices_z = img.shape[2]
    sclices_y = img.shape[1]
    sclices_x = img.shape[0]
    for z in range(sclices_z):
      if img[:,:,z].max() != 0:
        images.append((i ,z, "flair/z"))
        images.append((i ,z, "t1/z"))
        images.append((i ,z, "t2/z"))
        images.append((i ,z, "t1ce/z"))

    for y in range(sclices_y):
      if img[:,y,:].max() != 0:
        images.append((i ,y, "flair/y"))
        images.append((i ,y, "t1/y"))
        images.append((i ,y, "t2/y"))
        images.append((i ,y, "t1ce/y"))


    for x in range(sclices_x):
      if img[x,:,:].max() != 0:
        images.append(( i ,x, "flair/x"))
        images.append(( i ,x, "t1/x"))
        images.append((i ,x, "t2/x"))
        images.append(( i ,x, "t1ce/x"))

    return images


def dice_loss(y_true, y_pred):
    """
    implementation for generalized dice score loss
    GDL = 1 - (2 * sum(intersection)/(sum(pred)+sum(true)))
    """

    # y_pred = tf.round(y_pred)
    intersection = torch.sum(y_true * y_pred)
    pred_sum = torch.sum( y_pred)
    true_sum = torch.sum( y_true)
    smooth = torch.ones_like(intersection)*1e-5
    return 1-((2*intersection+smooth)/(pred_sum+true_sum+smooth))


model_type = 'vit_h'
checkpoint = 'sam_vit_h_4b8939.pth'
sam_model = sam_model_registry[model_type](checkpoint=checkpoint).to('cuda')



embedding_path = "/content/BraTS_data"
ground_truth_path = "/content/drive/MyDrive/SSA"

data = get_cases_list(embedding_path, ground_truth_path)
print(len(data))

train_list = data[:int(len(data)*0.8)]
val_list = data[int(len(data)*0.8):]

class BraTSDataset(CacheDataset):

    def __init__(self, ground_truth_path, embedding_path, list_of_data, device='cuda'):
        """
        Args:
            embedding_path (string): Path to  images.
            masks_path (string): path to masks.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.ground_truth_path = ground_truth_path
        self.device = device
        self.list_of_data = list_of_data
        self.embedding_path = embedding_path

    def __len__(self):
      return len(self.list_of_data)

    def __getitem__(self, idx):
      case_name, slic, d = self.list_of_data[idx]

      img =  nib.load(self.ground_truth_path + "/" + case_name.split(".")[0] + "/"+ case_name.split(".")[0] +"-seg.nii.gz")
      img = img.get_fdata()

      dim =  d.split("/")[-1]
      if dim == "x":
        mask = img[slic,:,:]
      elif dim == "y":
        mask = img[:,slic,:]
      elif dim == "z":
        mask = img[:,:,slic]


      d_embedding = np.load(self.embedding_path + "/" + d + "/embeddings/" + case_name + ".npy")
      embedding = d_embedding[:,:,:, slic]
      embedding = torch.as_tensor(embedding, dtype=torch.float)#, device=self.device)

      #--------------------------------------------------
      # mask[mask>0] = 1
      mask = transform.resize(
                      mask,
                      (256, 256),
                      order=0,
                      preserve_range=True,
                      mode="constant",
                  )

      box = get_bounding_box(mask)
      box_np = np.array(box)
      sam_trans = ResizeLongestSide(sam_model.image_encoder.img_size)
      box = sam_trans.apply_boxes(box_np, (mask.shape[-2], mask.shape[-1]))
      box_torch = torch.as_tensor(box, dtype=torch.float)#, device=self.device)

      class_1 = np.zeros_like(mask)
      class_2 = np.zeros_like(mask)
      class_3 = np.zeros_like(mask)

      class_1[np.where(mask == 1)] = 1
      class_2[np.where(mask == 2)] = 1
      class_3[np.where(mask == 3)] = 1

      mask = mask = np.stack((class_1, class_2, class_3), axis = 0 )


      mask = torch.as_tensor(mask)#.to(self.device)
      # mask = mask[None,...]

      sample = {"embedding": embedding,
                  'mask': mask,
                  'box' : box_torch
                  }

      return sample


train = BraTSDataset(ground_truth_path, embedding_path, train_list)
val = BraTSDataset(ground_truth_path, embedding_path, val_list)


train_loader = ThreadDataLoader(train, batch_size= 2, shuffle=True, num_workers=2)
val_loader = ThreadDataLoader(val, batch_size= 2, shuffle=True, num_workers=2)

# print(next(iter(train_loader)))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



hprams = {
    "num_epochs": 5,
    "best_loss": 1e10,
    "model_save_path": "/content",
    "lr":1e-5,
    "weight_decay":0,
    "device": device,
    "train_dataloader": train_loader,
    "val_dataloader": val_loader
    }

def eval(val_dataloader, sam_model, seg_loss):
  loss = 0
  for step_val, data in enumerate(val_dataloader):
    embedding, mask, box = data["embedding"], data["mask"], data["box"]

    with torch.no_grad():
      with torch.cuda.amp.autocast():
        # embedding = sam_model.image_encoder(image)
        sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
            points=None,
            boxes=box.to(device),
            masks=None,
        )

        # predicted masks
        mask_predictions, _ = sam_model.mask_decoder(
        image_embeddings= embedding.to(device), # (B, 256, 64, 64)
        image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
        sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
        dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
        multimask_output=True,
        )

        loss += seg_loss(mask_predictions, mask.to(device))
  loss /= step_val+1
  return loss



def train(**kwargs):
    model_save_path = kwargs['model_save_path']
    num_epochs = kwargs['num_epochs']
    best_loss = kwargs['best_loss']
    lr = kwargs['lr']
    weight_decay = kwargs['weight_decay']
    device = kwargs['device']
    train_dataloader = kwargs['train_dataloader']
    # embedding_path = kwargs['embedding_path']
    val_dataloader = kwargs['val_dataloader']

    os.makedirs(model_save_path, exist_ok=True)

    sam_model.train()
    optimizer = monai.optimizers.Novograd(sam_model.mask_decoder.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, factor=0.25,  patience=5)
    seg_loss = monai.losses.DiceLoss(sigmoid=True, squared_pred=True, reduction='mean')


    for epoch in range(num_epochs):
        epoch_loss = 0
        # train
        for step, data in enumerate(tqdm(train_dataloader)):
            mask, box, embedding =  data["mask"], data["box"], data["embedding"]
            # embedding.to(device)

            with torch.no_grad():
              with torch.cuda.amp.autocast():

                sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
                    points=None,
                    boxes=box.to(device),
                    masks=None,
                )
            with torch.cuda.amp.autocast():
            # predicted masks
              mask_predictions, _ = sam_model.mask_decoder(
                image_embeddings= embedding.to(device), # (B, 256, 64, 64)
                image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
                sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
                dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
                multimask_output=True,
              )

              loss = seg_loss(mask_predictions, mask.to(device))
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()
              epoch_loss += loss.item()
              # break
        epoch_loss /= step+1
        scheduler.step(epoch_loss)
        val_loss = None
        if val_dataloader is not None: val_loss = eval(val_dataloader, sam_model, seg_loss)
        print(f'EPOCH: {epoch}, Loss: {epoch_loss}, Val loss {val_loss}')
        # wandb.log({"epoch": epoch, "loss": loss})

        # save the latest model checkpoint
        # torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_'+str(epoch)+'.pth'))

        # save the best model
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_best.pth'))




train(**hprams)

## 2d Data multi node

In [ ]:
# saved 2d data
print("2d multinode")
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import Subset

from tqdm import tqdm
from torchsummary import summary
import monai
from segment_anything import SamPredictor, sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide
# import wandb
import nibabel as nib
from segment_anything.utils.transforms import ResizeLongestSide
import torch.nn.functional as F
from skimage import transform, io, segmentation
from torch.utils.data.distributed import DistributedSampler


torch.manual_seed(2023)
np.random.seed(2023)



def get_bounding_box(mask):

  y_indices, x_indices = np.where(mask > 0)
  if len(y_indices) == 0:
    return [0,0,0,0]
  x_min, x_max = np.min(x_indices), np.max(x_indices)
  y_min, y_max = np.min(y_indices), np.max(y_indices)
  # add perturbation to bounding box coordinates
  H, W = mask.shape
  x_min = max(0, x_min - np.random.randint(0, 20))
  x_max = min(W, x_max + np.random.randint(0, 20))
  y_min = max(0, y_min - np.random.randint(0, 20))
  y_max = min(H, y_max + np.random.randint(0, 20))
  # bboxes = np.array([x_min, y_min, x_max, y_max])

  return [x_min, y_min, x_max, y_max]




model_type = 'vit_h'
checkpoint = 'sam_vit_h_4b8939.pth'
sam_model = sam_model_registry[model_type](checkpoint=checkpoint).to('cuda')



embedding_path = "/content/BraTS_data/flair/z/embeddings"
ground_truth_path = "/content/BraTS_data/mask/z"

data = sorted(os.listdir(embedding_path))
train_list = data[:int(len(data)*0.7)]
val_list = data[int(len(data)*0.7):]

class BraTSDataset(Dataset):

    def __init__(self, ground_truth_path, embedding_path, list_of_data, sam_model, device='cuda'):
        """
        Args:
            embedding_path (string): Path to  images.
            masks_path (string): path to masks.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.ground_truth_path = ground_truth_path
        self.device = device
        self.list_of_data = list_of_data
        self.embedding_path = embedding_path
        self.sam_model = sam_model

    def __len__(self):
        return len(self.list_of_data)

    def __getitem__(self, idx):


      embedding = np.load(self.embedding_path + "/" + self.list_of_data[idx])
      embedding = torch.as_tensor(embedding, dtype=torch.float).cuda(gpu)


      #--------------------------------------------------

      mask = np.load(self.ground_truth_path + "/" + self.list_of_data[idx])
      mask[mask>0] = 1
      box = get_bounding_box(mask)
      # mask = squarify(mask)



      mask = transform.resize(
                      mask,
                      (256, 256),
                      order=0,
                      preserve_range=True,
                      mode="constant",
                  )


        # class_1 = np.zeros_like(mask)
        # class_2 = np.zeros_like(mask)
        # class_3 = np.zeros_like(mask)

        # class_1[np.where(mask == 1)] = 1
        # class_2[np.where(mask == 2)] = 1
        # class_3[np.where(mask == 4)] = 1

        # mask = mask = np.stack((class_1, class_2, class_3), axis = -1 )
      mask = torch.as_tensor(mask).cuda(gpu)
      mask = mask[None,...]


      box = get_bounding_box(mask)
      # box = [0,0,mask.shape[-2],mask.shape[-1]]
      box_np = np.array(box)
      sam_trans = ResizeLongestSide(self.sam_model.image_encoder.img_size)
      box = sam_trans.apply_boxes(box_np, (mask.shape[-2], mask.shape[-1]))
      box_torch = torch.as_tensor(box, dtype=torch.float).cuda(gpu)


      with torch.no_grad():
      # embedding = sam_model.image_encoder(image)
        sparse_embeddings, dense_embeddings = self.sam_model.prompt_encoder(
          points=None,
          boxes=box_torch,
          masks=None,
      )


      sample = {'embedding': embedding,
                'mask': mask,
                'image_pe' : sam_model.prompt_encoder.get_dense_pe(),
                'sparse_embeddings': sparse_embeddings,
                'dense_embeddings': dense_embeddings,
                  }

      return sample




def eval(val_dataloader, sam_model, seg_loss, gpu):
  loss = 0
  for step_val, data in enumerate(val_dataloader):
    embedding, mask, image_pe, sparse_embeddings, dense_embeddings = data["embedding"], data["mask"], data["image_pe"], data["sparse_embeddings"], data["dense_embeddings"]


    with torch.no_grad():
      # predicted masks
      mask_predictions, _ = sam_model(
      image_embeddings= embedding.cuda(gpu), # (B, 256, 64, 64)
      image_pe=image_pe.cuda(gpu), # (1, 256, 64, 64)
      sparse_prompt_embeddings=sparse_embeddings.cuda(gpu), # (B, 2, 256)
      dense_prompt_embeddings=dense_embeddings.cuda(gpu), # (B, 256, 64, 64)
      multimask_output=False,
      )

      loss += seg_loss(mask_predictions, mask.cuda(gpu))
  loss /= step_val+1
  return loss



def train( gpu, args):

    hprams = {
      "num_epochs": 15,
      "best_loss": 1e10,
      "model_save_path": "/content",
      "lr":1e-5,
      "weight_decay":0,

    }

    ngpus_per_node = torch.cuda.device_count()

    # Get the rank of the current process
    SLURM_NODEID = int(os.environ.get("SLURM_NODEID"))

    # Get the local ID of the current process
    SLURM_LOCALID = int(os.environ.get("SLURM_LOCALID"))

  #------------------------------------
    rank = SLURM_NODEID * SLURM_NODEID + SLURM_LOCALID
    dist.init_process_group(
    backend=args.backend,
    init_method=args.init_method,
    world_size=args.world_size,
    rank=rank
  )

  #-------------------------------------

    torch.cuda.set_device(gpu)


    model_type = 'vit_h'
    checkpoint = 'sam_vit_h_4b8939.pth'
    sam_model = sam_model_registry[model_type](checkpoint=checkpoint).cuda(gpu)

    train = BraTSDataset(ground_truth_path, embedding_path, train_list,sam_model , gpu)
    val = BraTSDataset(ground_truth_path, embedding_path, val_list, sam_model, gpu)

    train_sampler = DistributedSampler(train, num_replicas=args.world_size, rank=rank, shuffle=False, drop_last=False)
    val_sampler = DistributedSampler(val, num_replicas=args.world_size, rank=rank, shuffle=False, drop_last=False)


    train_loader = DataLoader(train, batch_size= 8, shuffle=False, num_workers=0, sampler=train_sampler)
    val_loader = DataLoader(val, batch_size= 4, shuffle=False, num_workers=0, sampler=val_sampler)

    model_save_path = hprams['model_save_path']
    num_epochs = hprams['num_epochs']
    best_loss = hprams['best_loss']
    lr = hprams['lr']
    weight_decay = hprams['weight_decay']


    os.makedirs(model_save_path, exist_ok=True)

    model = sam_model.mask_decoder
    model.cuda(gpu)


    model.train()
    model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[gpu])

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, factor=0.25,  patience=5)
    seg_loss = monai.losses.DiceLoss(sigmoid=True, squared_pred=True, reduction='mean').cuda(gpu)


    for epoch in range(num_epochs):
        epoch_loss = 0
        # train
        for step, data in enumerate(tqdm(train_dataloader)):
            embedding, mask, image_pe, sparse_embeddings, dense_embeddings = data["embedding"], data["mask"], data["image_pe"], data["sparse_embeddings"], data["dense_embeddings"]

            # with torch.no_grad():

            #   sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
            #       points=None,
            #       boxes=box,
            #       masks=None,
            #   )

            # predicted masks
            mask_predictions, _ = model(
              image_embeddings= embedding.cuda(gpu), # (B, 256, 64, 64)
              image_pe=image_pe.cuda(gpu), # (1, 256, 64, 64)
              sparse_prompt_embeddings=sparse_embeddings.cuda(gpu), # (B, 2, 256)
              dense_prompt_embeddings=dense_embeddings.cuda(gpu), # (B, 256, 64, 64)
              multimask_output=False,
            )

            loss = seg_loss(mask_predictions, mask.cuda(gpu))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            # break
        epoch_loss /= step+1
        scheduler.step(epoch_loss)
        val_loss = None
        if val_dataloader is not None: val_loss = eval(val_dataloader, model, seg_loss, gpu)
        print(f'EPOCH: {epoch}, Loss: {epoch_loss}, Val loss {val_loss}')
        # wandb.log({"epoch": epoch, "loss": loss})

        # save the latest model checkpoint
        # torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_'+str(epoch)+'.pth'))

        # save the best model
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.module.state_dict(), os.path.join(model_save_path, 'sam_model_best.pth'))



def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('-n', '--nodes', default=1,
                        type=int, metavar='N')
    parser.add_argument('-g', '--gpus', default=1, type=int,
                        help='number of gpus per node')
    parser.add_argument('-nr', '--nr', default=0, type=int,
                        help='ranking within the nodes')



    args = parser.parse_args()

    args.world_size = args.gpus * args.nodes
    os.environ['MASTER_ADDR'] = ''
    os.environ['MASTER_PORT'] = '8888'
    mp.spawn(train, nprocs=args.gpus, args=(args,))


if __name__ == "__main__":
  main()

In [1]:
def a():
  def b():
    print("b")


a.b()

AttributeError: ignored